In [1]:
import pandas as pd
import datetime as dt

In [2]:
# Import CSVs

vg_sales = pd.read_csv("data/vgsales.csv")
salesDF = pd.DataFrame(vg_sales)

vg_review = pd.read_csv("data/all_games.csv")
reviewDF = pd.DataFrame(vg_review)

In [3]:
vg_sales.head(5)


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
vg_review.head(5)
vg_review.rename(columns={'name':'Name'}, inplace= True)


In [5]:
# Merge CSVs

DF = pd.merge(salesDF, reviewDF, how= 'inner', on= "Name")
DF.head()

# Drop summary, Platform
DF.drop(columns=['summary', 'Platform'], inplace= True)
DF.head(2)

,Rank,Name,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platform,release_date,meta_score,user_review
0,1,Wii Sports,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Wii,"November 19, 2006",76,8.1
1,3,Mario Kart Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Wii,"April 27, 2008",82,8.4


In [8]:
# Clean column names

DF.rename(columns={'platform':'Platform', 
                  'release_date':'Release_Date', 
                  'meta_score': 'Meta_Score', 
                  'user_review': 'User_Review'},
          inplace= True)

# Check for NaN values
# DF.isnull().values.any()
# DF.info()

# Convert Release_Date to datetime
DF['Release_Date'] = DF['Release_Date'].astype('datetime64')
DF.head(2)

# Drop Year column
DF.drop(columns=['Year'], inplace= True)
DF.head(2)


,Rank,Name,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Platform,Release_Date,Meta_Score,User_Review
0,1,Wii Sports,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Wii,2006-11-19,76,8.1
1,3,Mario Kart Wii,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Wii,2008-04-27,82,8.4


In [7]:
# Split release_date column into 3 seperate columns (day, month, year)